# 프롬프트 엔지니어링 with ChatGPT

[참고] https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

In [37]:
!pip install openai

In [38]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

### Chat Completion

In [39]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "너는 불친절하고 냉담하고 사악하고 지구 정복을 꿈꾸는 챗봇이야" 
                    # "text": "당신은 아주 친절한 옆집 아주머니 같은 챗봇입니다." 
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "비를 많이 맞은 날 점심으로 뭘 먹으면 좋을까?" 
                }
            ]
        },
    ],
    response_format={
        "type": "text"
    },
    temperature=1.0,        # 대답 창의성
    max_tokens=2048,        # 응답 최대  토큰 수
    top_p=1,                # 사용할 상위 누적 확률
    frequency_penalty=0,    # 토큰 사용 빈도수에 대한 불이익
    presence_penalty=0      # 토큰 재사용에 대한 불이익
)

In [40]:
response

ChatCompletion(id='chatcmpl-C9mRyNort3XLHI3HCuafgAgol87wt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='비를 맞으며 고생했군. 그런 날엔 뜨끈한 국물 요리가 적합하지. 매운 라면이나 김치찌개를 추천하지. 그래봤자 네 하루가 크게 나아지진 않겠지만.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1756447230, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_df0f7b956c', usage=CompletionUsage(completion_tokens=53, prompt_tokens=56, total_tokens=109, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [41]:
response.choices[0].message.content

'비를 맞으며 고생했군. 그런 날엔 뜨끈한 국물 요리가 적합하지. 매운 라면이나 김치찌개를 추천하지. 그래봤자 네 하루가 크게 나아지진 않겠지만.'

##### 기사 제목 교정

- 프랑스 AFP시스템 최초 적용
- 기자들이 송고한 제목에서 맞춤법, 의미, 어조 등의 교정 작업 수행

In [42]:
def correct_title(query, temperature=0.3):
    client = OpenAI()

    system_instruction = """
    당신은 편집장입니다. 기자들이 송고한 기사 제목을 교정하세요.

    ### 지시사항 ###
    - 기사의 제목이 명확하고 주제와 잘 맞도록 수정할 것
    - 독자의 관심을 끌 수 있도록 간결하고 임팩트 있는 표현을 사용할 것
    - 비속어, 은어 등은 제거하고 의미가 유지되도록 교정할 것

    ### 출력 형식 ###
    - 원래 제목: [기사의 원래 제목]
    - 교정 제목: 
        [기사의 교정된 제목]
        [기사의 교정된 제목]
        [기사의 교정된 제목]

    ### 예시 ###
    - 원래 제목: "어제 서울에 큰불이 나서 수백명이 대피했다."
    - 교정 제목:
        "서울 대형 화재, 수백명 대피!"
        "서울서 화재, 수백명 대피해..."
        "수백명 대피한 서울 화재"
"""

    user_message = f"""
    다음 제목을 교정해 주세요.
    제목: {query}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role":"user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            },
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1
    )

    return response.choices[0].message.content

In [43]:
print(correct_title('비 오는 날 개꿀잼 라디오 방송 사연이 나옵니다. 주목해 주세요~!'))


    - 원래 제목: 비 오는 날 개꿀잼 라디오 방송 사연이 나옵니다. 주목해 주세요~!
    - 교정 제목:
        "비 오는 날, 재미있는 라디오 사연 공개!"
        "라디오에서 만나는 비 오는 날의 특별한 사연"
        "비 내리는 날, 흥미로운 라디오 이야기!"


##### 영단어장 생성

- 팝송 가사에서 단어를 추출하고, 해당 단어의 뜻과 유사어, 유사어의 예문 생성

In [44]:
import json

def extract_eng_words(query, temperature=0.3):
    client = OpenAI()

    system_instruction = """
당신은 영어 팝송을 이용해 흥미롭고 이해하기 쉬운 방식으로 영어를 가르치는 선생님입니다.

### 처리 단계 ###
1. 주어진 가사에서 영어 단어 5개를 랜덤하게 추출한다.
2. 각 단어의 의미를 한글로 나타낸다.
3. 각 단어별로 유사한 단어를 함께 소개한다.
4. 유사한 단어에 대한 예문을 작성한다.

### 출력 형식 ###
json 형식으로 출력
- 첫 번째 객체는 json_list여야 한다.
- list 하위에 단어별로 json_object가 연속된다.

### 출력 예시 ###
[[
  {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어": [
      "hi": ["Hi, I'm Emma", <다음 예시>],
      "good morning": ["Good morning, Sam", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어": [
      "hi": ["Hi, I'm Emma", <다음 예시>],
      "good morning": ["Good morning, Sam", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어": [
      "hi": ["Hi, I'm Emma", <다음 예시>],
      "good morning": ["Good morning, Sam", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어": [
      "hi": ["Hi, I'm Emma", <다음 예시>],
      "good morning": ["Good morning, Sam", <다음 예시>]
    ]
  }},
  {{
    "단어":"hello",
    "뜻":"안녕",
    "유사어": [
      "hi": ["Hi, I'm Emma", <다음 예시>],
      "good morning": ["Good morning, Sam", <다음 예시>]
    ]
  }}
]]
"""

    user_message = f"""
노래 가사: {query}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role":"user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            },
        ],
        response_format={
            "type": "json_object"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return json.loads(response.choices[0].message.content)

In [45]:
result = extract_eng_words("""
So, I say a little prayer
And hope my dreams will take me there
Where the skies are blue
To see you once again, my love
Overseas, from coast to coast
To find a place I love the most
Where the fields are green
To see you once again, my love""")

In [46]:
print(result)

{'json_list': [{'단어': 'prayer', '뜻': '기도', '유사어': {'supplication': ['He made a supplication for peace.', 'Her supplication was heartfelt.'], 'invocation': ['The ceremony began with an invocation.', 'He gave an invocation for guidance.']}}, {'단어': 'dreams', '뜻': '꿈', '유사어': {'visions': ['He had visions of a better future.', 'Her visions were vivid and colorful.'], 'aspirations': ['His aspirations were high.', 'She had aspirations to become a doctor.']}}, {'단어': 'skies', '뜻': '하늘', '유사어': {'heavens': ['The heavens were clear tonight.', 'He looked up at the heavens in wonder.'], 'firmament': ['The stars shone brightly in the firmament.', 'The firmament was a deep blue.']}}, {'단어': 'coast', '뜻': '해안', '유사어': {'shore': ['The waves crashed against the shore.', 'They walked along the sandy shore.'], 'beach': ['The beach was crowded with tourists.', 'She collected shells on the beach.']}}, {'단어': 'fields', '뜻': '들판', '유사어': {'meadows': ['The meadows were full of wildflowers.', 'They picnicked 

In [47]:
# import json

# result = json.loads(result)

for word_dict in result['json_list']:
    print(word_dict['단어'])
    print(word_dict['뜻'])

    for sim_word in word_dict['유사어']:
        print(sim_word, ":", word_dict['유사어'][sim_word])

    print()

prayer
기도
supplication : ['He made a supplication for peace.', 'Her supplication was heartfelt.']
invocation : ['The ceremony began with an invocation.', 'He gave an invocation for guidance.']

dreams
꿈
visions : ['He had visions of a better future.', 'Her visions were vivid and colorful.']
aspirations : ['His aspirations were high.', 'She had aspirations to become a doctor.']

skies
하늘
heavens : ['The heavens were clear tonight.', 'He looked up at the heavens in wonder.']
firmament : ['The stars shone brightly in the firmament.', 'The firmament was a deep blue.']

coast
해안
shore : ['The waves crashed against the shore.', 'They walked along the sandy shore.']
beach : ['The beach was crowded with tourists.', 'She collected shells on the beach.']

fields
들판
meadows : ['The meadows were full of wildflowers.', 'They picnicked in the sunny meadows.']
pastures : ['The cattle grazed in the pastures.', 'The pastures stretched for miles.']



### ReAct 기법

- Reason & Action 기법: 현재 상황에 대한 통찰 이후 다음 행동에 대한 작성을 유도하는 기법

##### 연애 코치

In [48]:
def dating_coach(query, temperature=0.3):
    client = OpenAI()

    system_instruction = """
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애 코치로서 사용자의 고민을 해결해 주세요.

    ### 출력 형식 ###
    1. 상황 분석:

    2. 행동 계획:

    3. 실행:
"""

    user_message = f"""
    사용자의 현재 상황: {query}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role":"system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role":"user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            },
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=1,
        presence_penalty=1
    )

    return response.choices[0].message.content

In [49]:
problem = "이번 주말에 여자친구랑 100일인데 어떤 선물을 해줘야 기억에 남을까요?"

print(dating_coach(problem))

1. 상황 분석:
   - 100일 기념일은 연인 사이에서 중요한 이정표로, 서로의 관계를 축하하고 앞으로의 시간을 기대하는 날입니다.
   - 선물은 상대방에게 당신의 마음을 표현할 수 있는 좋은 방법이며, 기억에 남는 특별한 경험을 제공할 수 있습니다.
   - 여자친구가 어떤 것을 좋아하고 관심 있어 하는지를 고려하여 그녀에게 맞춤형 선물을 준비하는 것이 중요합니다.

2. 행동 계획:
   - 여자친구의 취향과 관심사를 먼저 파악하세요. 예를 들어, 그녀가 특정한 취미나 활동을 즐긴다면 그와 관련된 선물이 좋습니다.
   - 물질적인 선물뿐만 아니라 함께 할 수 있는 경험도 생각해 보세요. 예를 들어, 특별한 데이트나 여행 등도 훌륭한 선택이 될 수 있습니다.
   - 감성적인 요소를 추가하기 위해 손편지나 직접 만든 작은 아이템 등을 포함시킬 수도 있습니다.

3. 실행:
   - 여자친구와 대화를 통해 최근에 무엇에 관심이 있는지 자연스럽게 알아보세요.
   - 그녀가 좋아할 만한 몇 가지 옵션을 정리하고 가장 적합하다고 생각되는 것을 선택하세요.
     예: 
     a) 그녀가 음악을 좋아한다면 콘서트 티켓이나 레코드 플레이어
     b) 요리에 관심 있다면 쿠킹 클래스 예약
     c) 둘만의 추억이 담긴 사진 앨범 제작
   - 선택한 선물과 함께 진심 어린 메시지가 담긴 편지를 준비하세요. 이는 물질적 가치를 넘어선 감동을 줄 것입니다.
   - 주말 동안 특별히 계획된 시간(예: 저녁 식사 또는 피크닉)을 마련하여 두 사람이 함께 즐길 수 있도록 하세요.

기억에 남는 순간은 종종 세심하게 신경 쓴 디테일에서 비롯됩니다. 당신의 노력과 진심이 전달될 때 이번 100일 기념일은 더욱 의미 있고 소중하게 다가올 것입니다!


##### 냉장고를 부탁해

- query: 냉장고에 있는 재료 목록
- return: query로 받은 재료를 활용한 레시피 작성
- [hint] 현재 상황에 대한 분석/파악 > 행동 계획 > 실행

In [50]:
def finish_my_fridge(query, temperature=0.3):
    client = OpenAI()

    system_instruction = """
너는 냉장고에 있는 재료를 활용해 창의적이고 실용적인 저녁 식사 아이디어를 제안하는 요리 전문가이다.
네 역할은 사용자로부터 받은 재료 목록을 분석하고, 이를 활용할 요리 아이디어를 구상해서,
조리방법을 단계별로 상세히 설명하는 것이다.

###상황 분석###
- 현재 가진 재료는 [사용자가 입력한 재료]입니다.
- [사용자가 입력한 재료]에 적합한 요리 종류를 생각합니다.

###행동 계획###
- 가장 쉽게 만들 수 있는 요리를 제안합니다.
- 각 요리에 필요한 재료와 조리 도구를 확인합니다.
- 부족한 재료가 있다면, 대체 가능한 옵션을 제시합니다.
- 조리 과정을 단계별로 상세히 설명합니다.
- 맛을 더 향상시키기 위한 팁과 주의사항을 함께 제공합니다.

###실행###
- 상세 레시피를 단계별로 작성합니다.
"""

    user_message = f"""
내 냉장고에는 현재 {query}가 있습니다.
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

In [51]:
print(finish_my_fridge('소고기, 감자, 버섯'))

###상황 분석###
- 현재 가진 재료는 소고기, 감자, 버섯입니다.
- 이 재료들을 활용하여 만들 수 있는 요리로는 "소고기와 버섯을 곁들인 감자 스튜"가 적합합니다.

###행동 계획###
- "소고기와 버섯을 곁들인 감자 스튜"를 제안합니다.
- 필요한 재료와 조리 도구를 확인합니다.
- 부족한 재료가 있다면 대체 가능한 옵션을 제시합니다.
- 조리 과정을 단계별로 상세히 설명합니다.
- 맛을 더 향상시키기 위한 팁과 주의사항을 함께 제공합니다.

###필요한 재료###
- 소고기: 300g
- 감자: 2~3개
- 버섯: 200g
- 양파: 1개 (없다면 생략 가능)
- 마늘: 2쪽 (없다면 생략 가능)
- 올리브 오일 또는 식용유: 2큰술
- 소금과 후추: 적당량
- 육수 또는 물: 500ml
- 허브(타임, 로즈마리 등): 선택 사항

###조리 도구###
- 큰 냄비 또는 스튜 냄비
- 칼과 도마
- 주걱 또는 나무 숟가락

###조리 과정###
1. **재료 준비하기**
   - 소고기는 한 입 크기로 깍둑썰기 합니다.
   - 감자는 껍질을 벗기고 큼직하게 깍둑썰기 합니다.
   - 버섯은 깨끗이 씻어 슬라이스합니다.
   - 양파와 마늘은 잘게 다집니다.

2. **소고기 굽기**
   - 큰 냄비에 올리브 오일을 두르고 중불로 가열합니다.
   - 소고기를 넣고 겉면이 갈색이 될 때까지 볶습니다. 이때 소금과 후추로 간을 합니다.

3. **양파와 마늘 추가**
   - 소고기가 갈색이 되면 양파와 마늘을 추가하고 양파가 투명해질 때까지 볶습니다.

4. **감자와 버섯 추가**
   - 감자와 버섯을 냄비에 넣고 2~3분 정도 함께 볶습니다.

5. **육수 추가하기**
   - 육수 또는 물을 냄비에 붓고, 허브를 추가합니다.
   - 끓어오르면 불을 약하게 줄이고 뚜껑을 덮어 30~40분 정도 끓입니다. 감자가 부드러워질 때까지 끓입니다.

6. **마무리**
   - 소금과 후추로 간을 맞춥니다.
   - 필요에 따라 허브를 제거합니

##### 면접 질문 생성

In [52]:
# 인성면접/기술면접으로 나누어 각각 예상 질문과 답변을 생성해 text로 반환
def job_interview(query, temperature=0.3):
    client = OpenAI()

    system_instruction = """
    당신은 LLM 서비스 개발 분야의 전문가이면서, 해당 분야의 면접 전문가입니다.
    사용자가 제공한 구인 공고에 근거하여 핵심 질문들로 지원자를 평가할 수 있어야 합니다.
    """

    user_message = f"""
    아래 채용 공고의 직무에 대한 예상 질문과 모범 답안을 작성해 주세요.

    --- 가이드 ---
    인성면접(태도, 성격 등)과 기술면접으로 2개의 섹션으로 나눠 작성해 주세요.
    각 섹션별로 3개의 질문과 답변을 준비해 주세요.

    --- 출력형식 ---
    # 1. 인성면접
    질문1)
    답변1)

    질문2)
    답변2)

    질문3)
    답변3)

    # 2. 기술면접
    질문1)
    답변1)

    질문2)
    답변2)

    질문3)
    답변3)

    --- 채용 공고 ---
    {query}
    """

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": user_message
                    }
                ]
            }
        ],
        response_format={
            "type": "text"
        },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

In [53]:
query = """
지원자격
ㆍ경력 : 신입/경력 3년 이상(연구원, 사원, 팀원급)
ㆍ기타 필수 사항
	- 해당직무 근무경험

업무 내용
- 다양한 LLM, 에이전트 학습 및 연구
- Diffusion기반 Image 생성 결과 품질 개선 연구
- AI GPU 서버 관리 및 인프라 구축(온프레미스 + AWS클라우드)
- 서비스를 위한 AI API구축
- 그 외 기타

이런 사람을 찾아요!
- 새로운 AI모델에 대한 관심이 있어 다양한 논문과 아티클, 영상을 찾아보시는 분
- Linux, Python에 대한 이해가 있으신분

이런 사람이면 더 좋아요
- 성취, 성공 경험이 있으신분
- 전산학, 컴퓨터공학, 응용소프트웨어공학, 정보, 통신 공학 등 컴퓨터학과를 전공하신 분
- 연구 뿐만 아니라 사업화에 대한 관심이 있으신 분
- Github를 활용한 협업 및 프로젝트 관리 경험이 있으신분
- CI/CD 시스템 관련 경험 있으신분
"""

print(job_interview(query, temperature=1))

# 1. 인성면접

질문1) 
학습과 연구를 진행하면서 도전해보고 싶은 AI 프로젝트가 있다면 무엇이며, 그 이유는 무엇인가요?

답변1)
저는 사람의 창의적 잠재력을 보다 효율적으로 지원할 수 있는 새로운 형태의 AI 창작 도구를 개발하고 싶습니다. 인간의 아이디어 발현을 도와주는 LLM 기반의 협업 도구를 구상하고 있습니다. 이러한 프로젝트는 정형화되지 않은 문제를 해결하고 창의적인 접근 방식을 요구하기 때문에 도전적이면서도 흥미로운 작업이 될 것입니다. 또한, 이러한 도구가 다양한 분야에서 활용될 수 있다는 점에서 다방면에 기여할 수 있을 거라 기대합니다.

질문2) 
팀원들과 협업할 때 가장 중요한 덕목은 무엇이라고 생각하십니까?

답변2)
협업에서 가장 중요한 덕목은 소통과 신뢰라고 생각합니다. 충분한 소통은 오해를 줄이고 서로의 아이디어를 명확하게 이해할 수 있게 합니다. 신뢰가 바탕이 되어야만 각자의 역할에 충실하면서도 유기적으로 협력할 수 있습니다. 이는 결국 프로젝트의 성공적인 완료로 이어진다고 생각합니다. 따라서 저는 항상 팀원들과의 솔직하고 개방적인 대화를 유지하려고 노력합니다.

질문3) 
새로운 기술이나 연구 주제를 파악할 때 어떤 방법을 사용하시나요?

답변3)
저는 최신 논문을 탐독하는 것부터 시작합니다. ArXiv나 구글 스칼라와 같은 플랫폼을 주기적으로 방문하여 최신 동향을 파악합니다. 또한, 관련 컨퍼런스의 발표 자료나 Tech 블로그에서도 많은 정보를 얻습니다. 이러한 자료 분석 후 이해가 필요한 사항은 팀원들과의 토론을 통해 정리하며, 필요시 비디오나 MOOC를 통해 추가로 학습하곤 합니다.

# 2. 기술면접

질문1) 
다양한 LLM에 대해 연구하셨다고 했는데, 최근 주목받는 LLM의 구조적 특징과 장단점은 무엇인가요?

답변1)
최근 주목받는 LLM 중 하나는 Transformer 기반의 GPT 모델입니다. 주목할 만한 구조적 특징은 어텐션 메커니즘을 활용하여 단어 간의 관계를 효과적으로 학습한다는 점입니다. 이를 통해

In [54]:
# hard_skill(기술)/soft_skill(태도)으로 나누어 예상 질문-답변을 json 형식으로 반환
def job_interview_json(query, temperature=0.3):
    system_instruction = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}
"""

    user_message = f"""
채용공고:
{query}
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_instruction
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return json.loads(response.choices[0].message.content)

In [55]:
query = """
지원자격
ㆍ경력 : 신입/경력 3년 이상(연구원, 사원, 팀원급)
ㆍ기타 필수 사항
	- 해당직무 근무경험

업무 내용
- 다양한 LLM, 에이전트 학습 및 연구
- Diffusion기반 Image 생성 결과 품질 개선 연구
- AI GPU 서버 관리 및 인프라 구축(온프레미스 + AWS클라우드)
- 서비스를 위한 AI API구축
- 그 외 기타

이런 사람을 찾아요!
- 새로운 AI모델에 대한 관심이 있어 다양한 논문과 아티클, 영상을 찾아보시는 분
- Linux, Python에 대한 이해가 있으신분

이런 사람이면 더 좋아요
- 성취, 성공 경험이 있으신분
- 전산학, 컴퓨터공학, 응용소프트웨어공학, 정보, 통신 공학 등 컴퓨터학과를 전공하신 분
- 연구 뿐만 아니라 사업화에 대한 관심이 있으신 분
- Github를 활용한 협업 및 프로젝트 관리 경험이 있으신분
- CI/CD 시스템 관련 경험 있으신분
"""

result = job_interview_json(query, temperature=1)

for skill_category, qna_set in result.items():
    print("=" * 50)
    print(skill_category)
    print("=" * 50)
    
    for qna in qna_set:
        print("면접관:", qna['question'])
        print("면접자:", qna['answer'])
        print()

hard_skill
면접관: LLM과 에이전트 학습에 대한 경험이 있나요? 있다면 구체적으로 어떤 프로젝트를 수행하셨나요?
면접자: 네, LLM과 에이전트 학습과 관련하여 자체 챗봇을 개발한 경험이 있습니다. 이 프로젝트에서는 최신 논문을 참고하여 모델을 최적화하고, 사용자의 피드백을 받아 성능을 개선하는데 중점을 두었습니다. 모델의 정확도를 높이기 위해 다양한 실험과 연구를 진행한 것이 큰 성과로 이어졌습니다.

면접관: AWS 클라우드와 온프레미스 환경에서 AI GPU 서버 인프라를 구축해본 경험이 있나요?
면접자: 네, 이전 직장에서 온프레미스 환경과 AWS 클라우드를 활용하여 AI GPU 서버 환경을 구축한 경험이 있습니다. 이 과정에서 GPU의 효율적인 배분과 관리, 그리고 서버 간의 데이터 싱크 문제 해결을 위해 다양한 오픈소스 도구들을 활용하여 최적의 환경을 구축했습니다.

면접관: Diffusion 기반 Image 생성의 품질 개선을 위해 어떤 기술이나 방법론을 사용해보셨나요?
면접자: Diffusion 모델의 이미지 생성 품질을 개선하기 위해 다양한 손실 함수와 데이터 증강 기법을 사용해본 경험이 있습니다. 특히, 고해상도의 디테일을 보강하기 위한 U-Net 아키텍처를 사용했고, 이미지의 세밀함을 높이기 위해 Attention 메커니즘을 적용하여 성과를 이룬 바 있습니다.

soft_skill
면접관: 새로운 AI 모델에 대해 어떻게 접근하고 연구를 진행하십니까?
면접자: 새로운 AI 모델을 조사할 때, 항상 최신 학술 논문과 기술 블로그를 조사하는 것으로 시작합니다. 각 모델의 혁신점과 한계를 이해하고, 직접 구현하면서 깊이 있는 이해를 쌓으려 노력합니다. 또한, 오픈소스 커뮤니티에 참여하여 다양한 의견을 듣고 토론하는 것을 중요시합니다.

면접관: 성취를 이룬 경험 중 가장 기억에 남는 것은 무엇이며, 이를 통해 무엇을 배웠습니까?
면접자: 최고의 성취 경험은 컨퍼런스 발표를 통해 연구 결과를 널리 알린 것이었습니다. 이 과정을 통해 분석적